**TODO**
1. Map the note names to a MIDI number like scale for analyzing intervals
2. Relation between measures are indicated how? (Extensions, glissando, my vocab is poor here)
3. Multi-part scores. Probably not possible in makam music

**Questions**
- XML
    - What is alter? I think numeric value indicating accidental?
    - What is "divisions" attribute?
- Makam Music
    - Can makam music have time signature change?

```Measure
    0: attributes ( even if no note is present) 
        sometimes empty, o.w.
        'divisions'
        'key'
        'time'
    1,.. notes ```

In [1]:
import os

import numpy as np

import xml.etree.ElementTree as ET

In [6]:
# Improved from https://github.com/burakuyar/Tools/blob/master/musicxml_player.py

def parse_notes(root, record_embellishment=True):
    """
    Returns a 2D array of [[measure_idx,note_idx,note_duration,note_name]]
    Note name can only be "PitchClass Octave", "PitchClass Octave Accidental" or "Rest"
    If a note_name is not Rest and it has zero duration, that note is an embellishment.
    """
    notes=[]
    for m_idx,measure in enumerate(root.findall('part/measure')):   
        if len(measure.findall('note'))>0: # Check if the measure contains at least one note
            grace_count=0 # Count grace notes in case you don't want to record them
            for n_idx,note in enumerate(measure.findall('note')):
                dur=note.find('duration')
                if dur is None:
                    if not record_embellishment:
                        grace_count+=1
                        continue # skip the grace note
                    else:
                        dur='0' # Embellishment/Grace Note
                else:
                    dur=dur.text
                step=note.find('pitch/step')
                if step is not None:
                    step=step.text
                    octave=note.find('pitch/octave').text
                    acc = note.find('accidental')
                    if acc is None:
                        n='{}{}'.format(step, octave)
                    else:
                        n='{}{} {}'.format(step, octave, acc.text)
                else:
                    rest = note.find('rest')
                    assert rest is not None, "The note doesn't have a pitch and is not a rest!"
                    n='Rest'
                note = [m_idx, n_idx-grace_count, dur, n]
                notes.append(note)
    return np.array(notes)

def get_time_signatures(root):
    """Returns all time signatures in the score as a list of tupples.
    Assumes it is possible to have time change in makam pieces."""
    beats = [t.text for t in root.findall('part/measure/attributes/time/beats')]
    types = [t.text for t in root.findall('part/measure/attributes/time/beat-type')]
    all_time_signatures=[(int(b),int(t)) for b,t in zip(beats,types)]
    return all_time_signatures

def get_bpm(root):
    return float(root.find('part/measure/direction/sound').attrib['tempo'])

def get_divisions(root):
    return float(root.find('part/measure/attributes/divisions').text)

def find_key_signature_accidentals(root):
    notes = []
    accidentals = []
    for k in root.iter('key'):
        for ks in k.findall('key-step'):
            notes.append(ks.text)
        for ka in k.findall('key-accidental'):
            accidentals.append(ka.text)
    return ['{} {}'.format(n,k) for n,k in zip(notes,accidentals)]

def find_all_accidentals(root):
    # Access accidental names as follows
    accidentals=[]
    for a in root.iter('accidental'):
        #print(a.text)
        accidentals.append(a.text)
    accidentals=set(accidentals)
    #print(accidentals)
    return accidentals
        
#qnotelen = 60000/bpm

# 1) Read an XML file

In [31]:
score_name='hicaz--sarki--aksaksemai--sezdim_dargin--rifat_ayaydin.xml'

data_dir=os.path.join(os.getcwd(), 'data')
score_path=os.path.join(data_dir, score_name)

tree=ET.parse(score_path)
root=tree.getroot()

#print(float(root.find('part/measure/attributes/divisions').text))

In [32]:
bpm=get_bpm(root)
print(f'BPM: {bpm}')
divs=get_divisions(root)
print('Divisions: {}'.format(divs))
key_signature_accidentals=find_key_signature_accidentals(root)
print('Accidentals in the key signature: {}'.format(key_signature_accidentals))
notes=parse_notes(root, record_embellishment=False)

BPM: 65.0
Divisions: 96.0
Accidentals in the key signature: ['B slash-flat', 'F sharp', 'C sharp']


In [33]:
#print('note_idx, note_duration, note_name')
#print('='*len('note_idx, note_duration, note_name'))
for n in notes:
    print(n)   

['0' '0' '48' 'G5']
['0' '1' '48' 'F5 sharp']
['0' '2' '24' 'G5']
['0' '3' '24' 'E5']
['0' '4' '96' 'E5']
['0' '5' '48' 'E5']
['0' '6' '24' 'F5 sharp']
['0' '7' '24' 'G5']
['0' '8' '24' 'A5']
['0' '9' '24' 'G5']
['0' '10' '24' 'F5']
['0' '11' '24' 'E5']
['0' '12' '48' 'D5']
['1' '0' '48' 'C5 sharp']
['1' '1' '48' 'B4 slash-flat']
['1' '2' '48' 'A4']
['1' '3' '72' 'D5']
['1' '4' '24' 'C5 sharp']
['1' '5' '24' 'C5 sharp']
['1' '6' '24' 'B4 slash-flat']
['1' '7' '24' 'B4 slash-flat']
['1' '8' '24' 'A4']
['1' '9' '144' 'A4']
['2' '0' '48' 'A4']
['2' '1' '48' 'B4 slash-flat']
['2' '2' '48' 'C5 sharp']
['2' '3' '48' 'E5']
['2' '4' '24' 'D5']
['2' '5' '24' 'C5 sharp']
['2' '6' '24' 'C5 sharp']
['2' '7' '24' 'B4 slash-flat']
['2' '8' '24' 'B4 slash-flat']
['2' '9' '24' 'A4']
['2' '10' '72' 'A4']
['2' '11' '24' 'G4']
['2' '12' '12' 'F5']
['2' '13' '12' 'E5']
['2' '14' '12' 'D5']
['2' '15' '12' 'E5']
['3' '0' '96' 'D5']
['3' '1' '48' 'E5']
['3' '2' '48' 'E5']
['3' '3' '48' 'D5']
['3' '4' '96' 'D